In [ ]:
import json

"""
with open('chatbot_respostas.json', encoding='utf8') as f:
    json_respostas = json.load(f)

with open('chatbot_respostas.json', 'w', encoding='utf8') as f:
    for entrada in json_respostas:
        resposta = {
            'resposta': entrada['resposta'],
            'categoria': entrada['categoria'],
            'fonte': 'ChampBot'
        }
        entrada_jsonl = {
            'messages': [
                {'role': 'user', 'content': entrada['pergunta']},
                {'role': 'assistant', 'content': json.dumps(resposta, ensure_ascii=False, indent=2)}
            ]
        }
        json.dump(entrada_jsonl, f, ensure_ascii=False)
        f.write('\n')"
"""

In [1]:
import openai
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv())
client = openai.Client()

In [ ]:
"""
file = client.files.create(
    file=open('chatbot_respostas.jsonl', 'rb'),
    purpose='fine-tune'
)

client.fine_tuning.jobs.create(
    training_file=file.id,
    model="gpt-3.5-turbo"
)
"""

FineTuningJob(id='ftjob-5FbLjaqX0xeRM8ceOqaumByC', created_at=1741793421, error=Error(code=None, message=None, param=None), fine_tuned_model=None, finished_at=None, hyperparameters=Hyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto'), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-nkjgDkDcPNtTnjoc6ctqKvIK', result_files=[], seed=1137124733, status='validating_files', trained_tokens=None, training_file='file-9MN2R3DTLFu7RzHTB3MvuE', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size='auto', learning_rate_multiplier='auto', n_epochs='auto')), type='supervised'), user_provided_suffix=None)

In [2]:
client.fine_tuning.jobs.list()

SyncCursorPage[FineTuningJob](data=[FineTuningJob(id='ftjob-5FbLjaqX0xeRM8ceOqaumByC', created_at=1741793421, error=Error(code=None, message=None, param=None), fine_tuned_model='ft:gpt-3.5-turbo-0125:personal::BAIXLd7l', finished_at=1741794233, hyperparameters=Hyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3), model='gpt-3.5-turbo-0125', object='fine_tuning.job', organization_id='org-nkjgDkDcPNtTnjoc6ctqKvIK', result_files=['file-VEm4U3QrmAYSZHBXb6Lgse'], seed=1137124733, status='succeeded', trained_tokens=29211, training_file='file-9MN2R3DTLFu7RzHTB3MvuE', validation_file=None, estimated_finish=None, integrations=[], metadata=None, method=Method(dpo=None, supervised=MethodSupervised(hyperparameters=MethodSupervisedHyperparameters(batch_size=1, learning_rate_multiplier=2.0, n_epochs=3)), type='supervised'), user_provided_suffix=None)], has_more=False, object='list')

In [3]:
mensagens = [{'role': 'user', 'content': 'O que é uma equação quadrática?'}]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

Uma equação quadrática é uma equação polinomial de segundo grau, ou seja, uma equação na forma ax^2 + bx + c = 0, onde "a", "b" e "c" são coeficientes reais e "x" é a variável desconhecida. A solução de uma equação quadrática pode ser encontrada utilizando a fórmula de Bhaskara ou completando o quadrado. As soluções de uma equação quadrática podem ser reais ou complexas.


In [4]:
system_mes = '''
Responda as perguntas em um parágrafo de até 20 palavras. Categorize as respostas no seguintes conteúdos: física, matemática, língua portuguesa ou outros.
Retorne a resposta em um formato json, com as keys:
fonte: valor deve ser sempre ChampBot
resposta: a resposta para a pergunta
categoria: a categoria da pergunta
'''

mensagens = [
    {'role': 'system', 'content': system_mes},
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='gpt-3.5-turbo',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
    "fonte": "ChampBot",
    "resposta": "Uma equação quadrática é uma equação polinomial de segundo grau, representada na forma ax^2 + bx + c = 0.",
    "categoria": "matemática"
}


In [5]:
resposta.usage

CompletionUsage(completion_tokens=61, prompt_tokens=112, total_tokens=173, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))

In [6]:
# usando o modelo treinado por fine-tuning para evitar o alto uso de tokens diminuindo assim os gastos de geração de respostas

mensagens = [
    {'role': 'user', 'content': 'O que é uma equação quadrática?'}
]

resposta = client.chat.completions.create(
    messages=mensagens,
    model='ft:gpt-3.5-turbo-0125:personal::BAIXLd7l',
    max_tokens=1000,
    temperature=0
)

print(resposta.choices[0].message.content)

{
  "resposta": "Uma equação quadrática é uma equação polinomial de segundo grau, geralmente na forma ax² + bx + c = 0.",
  "categoria": "Matemática",
  "fonte": "ChampBot"
}


In [7]:
resposta.usage

CompletionUsage(completion_tokens=60, prompt_tokens=16, total_tokens=76, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0))